# Пайплайн № 7
### База знаний: FAQ, Реальные Кейсы, Пользовательское соглашение и Условия размещения

### Защита от галлюцинаций: если не уверены в ответе - возвращаем "Нет ответа", но отдаём два классификатора.

### Добавили Ranker, который более тщательно ранжирует релевантные вопросы

### Добавили LLM

### Установка необходимых библиотек

In [1]:
# !export LLAMA_CUBLAS=1
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --verbose
# !pip install llama-cpp-haystack

In [2]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=3.0.0"
pip install pymorphy2
pip install catboost
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


### Подготовка данных

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r drive/MyDrive/HacksAI/Russia24/Dataset Dataset/

Эталонная База Знаний FAQ

In [5]:
import pandas as pd
import json

know_base_path = 'Dataset/01_База_знаний.xlsx'
df_base = pd.read_excel(know_base_path)

df_base.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


База Реальных Кейсов

In [6]:
real_case_path = 'Dataset/02_Реальные_кейсы.xlsx'
df_case = pd.read_excel(real_case_path)

df_case.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


Объединим обе базы для полее полного и объективного поиска

In [7]:
combined_column = pd.concat([df_base['Вопрос из БЗ'], df_case['Вопрос пользователя'], df_base['Ответ из БЗ']], ignore_index=True)

df_base_rest = df_base.drop(columns=['Вопрос из БЗ'])
df_case_rest = df_case.drop(columns=['Вопрос пользователя'])

combined_rest = pd.concat([df_base_rest, df_case_rest, df_base_rest], ignore_index=True)

final_df = combined_rest.copy()
final_df['Вопрос'] = combined_column

final_df = final_df[['Вопрос'] + [col for col in final_df.columns if col != 'Вопрос']]
final_df = final_df.drop(columns=["Тема"])

final_df.tail()

,Вопрос,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,Ответ сотрудника,Вопрос из БЗ
1448,Категория должна соответствовать содержанию ви...,Категория должна соответствовать содержанию ви...,ВИДЕО,Загрузка видео,NaN,NaN
1449,Если при загрузке видео вы выбирали одну катег...,Если при загрузке видео вы выбирали одну катег...,МОДЕРАЦИЯ,Смена категории/возрастные ограничения,NaN,NaN
1450,"Войдите в профиль на RUTUBE, перейдите в разде...","Войдите в профиль на RUTUBE, перейдите в разде...",ВИДЕО,Загрузка видео,NaN,NaN
1451,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео,NaN,NaN
1452,Главное — соблюдать несколько правил.\n\nТайм-...,Главное — соблюдать несколько правил.\n\nТайм-...,ВИДЕО,Загрузка видео,NaN,NaN


In [8]:
import docx
from haystack import Document

docs = [Document(content=row["Вопрос"], meta={"source": "FAQ", "idx": index}) for index, row in final_df.iterrows()]

Добавим документы: Пользовательское соглашение и Условия размещения

In [9]:
my_doc = docx.Document('Dataset/03_ГЕНЕРАЛЬНОЕ_ПОЛЬЗОВАТЕЛЬСКОЕ_СОГЛАШЕНИЕ_RUTUBE.docx')
para_list = [paragraph.text for paragraph in my_doc.paragraphs if paragraph.text != ""]

para_tuples_list = []

for paragraph in para_list:
    parts = paragraph.split("    ")
    if len(parts) > 1:
        point, content = parts[0].strip(), parts[-1].strip()
        # Разбиваем содержание на чанки по 500 символов
        for i in range(0, len(content), 2048):
            chunk = content[i:i+2048]
            para_tuples_list.append((point, chunk))


for index, paragraph in para_tuples_list:
  docs.append(Document(content=paragraph, meta={"source": "Agreement", "idx": index}))

In [10]:
my_doc = docx.Document('Dataset/04_УСЛОВИЯ РАЗМЕЩЕНИЯ КОНТЕНТА.docx')
para_list = [paragraph.text for paragraph in my_doc.paragraphs if paragraph.text != ""]

para_tuples_list = []

for paragraph in para_list:
    parts = paragraph.split(".")
    idx = parts[0].strip() if len(parts[0].strip()) <= 2 else ""
    content = ".".join(parts[1:]).strip() if idx else paragraph.strip()

    # Разбиваем содержание на чанки
    for i in range(0, len(content), 2048):
        chunk = content[i:i+2048]
        para_tuples_list.append((idx, chunk))

for index, paragraph in para_tuples_list:
  docs.append(Document(content=paragraph, meta={"source": "Content", "idx": index}))

In [11]:
len(docs)

1648

### Инициализируем Трансформер для построения эмбеддинга вопросов

In [12]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.utils import ComponentDevice, Device

device = ComponentDevice.from_str("cuda:0")

embed_model = "cointegrated/LaBSE-en-ru"
# embed_model = "intfloat/e5-large-v2"

doc_embedder = SentenceTransformersDocumentEmbedder(
    model=embed_model,
    device=device
    )

doc_embedder.warm_up()

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/516M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

### Запишем вопросы в векторное хранилище

In [13]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

docs_with_embeddings = doc_embedder.run(docs)
unique_docs = {doc.id: doc for doc in docs_with_embeddings["documents"]}.values()
document_store.write_documents(list(unique_docs))

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

1648

### Инициализируем Трансформер для поиска по эмбеддингам вопросов
он должен совпадать с трансформером, на основе которого строились эмбеддинги

In [14]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(
    model=embed_model,
    device=device,
)

### Инициализируем Retriever

In [15]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store, top_k=100)

### Добавим LLM модель, которой на вход мы будем подавать весь найденный в RAG контекст

In [16]:
from transformers import pipeline
GROUNDED_SYSTEM_PROMPT = "Your task is to answer the user's questions using only the information from the provided documents."

pipe = pipeline("text-generation", model="Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24", device="cuda:0")

config.json:   0%|          | 0.00/984 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

## Собираем пайплайн

In [17]:
from haystack import Pipeline

from haystack.components.rankers import TransformersSimilarityRanker

ranker = TransformersSimilarityRanker(
    model="DiTy/cross-encoder-russian-msmarco",
    top_k=3,
    device=device,
    model_kwargs = {"max_length": 512},
    tokenizer_kwargs={"model_max_length": 512}
    )

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("ranker", ranker)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "ranker")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - ranker: TransformersSimilarityRanker
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> ranker.documents (List[Document])

Добавим словарь, по которому будем заменять англицизмы на действительно английские слова (т. к. в базе они хранятся именно на английском)

In [18]:
replace_dict = {
    'шортс': 'shorts',
    'шортсы': 'shorts',
    'шортса': 'shorts',
    'шортсу': 'shorts',
    'шортсом': 'shorts',
    "рутуб": "rutube",
    "рутубе": "rutube",
    "рутуба": "rutube",
    "рутубу": "rutube",
    "ютуб": "youtube",
    "ютубе": "youtube",
    "ютуба": "youtube",
    "ютубу": "youtube",
    "смарт": "smart",
    "тв": "tv",
    "самсунг": "samsung",
    "урл": "url",
}

### Загрузим предварительно обученные модели для будущей классификации вопросов

In [19]:
!cp -r drive/MyDrive/HacksAI/Russia24/models/version_20240928_121531_train_0.80_test_0.66_valid_0.61/ model_1/
!cp -r drive/MyDrive/HacksAI/Russia24/models/version_20240928_123836_train_0.45_test_0.41_valid_0.32/ model_2/

Вспомогательные функции для моделей классификации

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import joblib

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

loaded_models = {}
loaded_encoders = {}
loaded_vectorizers = {}

def preprocess(text):
    text = re.sub(r'\W+', ' ', text)

    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)

def load_model_and_files(model_name, model_path, encoder_path, vectorizer_path):
    global loaded_models, loaded_encoders, loaded_vectorizers

    if model_name not in loaded_models:
        print(f"Загрузка модели {model_name}...")
        loaded_models[model_name] = joblib.load(model_path)
        loaded_encoders[model_name] = joblib.load(encoder_path)
        loaded_vectorizers[model_name] = joblib.load(vectorizer_path)
    else:
        print(f"Модель {model_name} уже загружена.")

def classify_question(
    question: str,
    model_name: str,
):
    global loaded_models, loaded_encoders, loaded_vectorizers

    preprocessed_question = preprocess(question)

    # Преобразование вопроса в TF-IDF вектор
    vectorizer = loaded_vectorizers[model_name]
    question_tfidf = vectorizer.transform([preprocessed_question])

    # Предсказание с помощью модели
    model = loaded_models[model_name]
    predicted_label_numeric = model.predict(question_tfidf)

    # Преобразование числовой метки обратно в текстовую метку
    encoder = loaded_encoders[model_name]
    predicted_label_text = encoder.inverse_transform(predicted_label_numeric)

    return predicted_label_text[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
model_path_1 = "model_1/classifier_1.pkl"
encoder_path_1 = "model_1/label_encoder_1.pkl"
vectorizer_path_1 = "model_1/vectorizer_1.pkl"

model_path_2 = "model_2/classifier_2.pkl"
encoder_path_2 = "model_2/label_encoder_2.pkl"
vectorizer_path_2 = "model_2/vectorizer_2.pkl"

load_model_and_files('model_1', model_path_1, encoder_path_1, vectorizer_path_1)
load_model_and_files('model_2', model_path_2, encoder_path_2, vectorizer_path_2)

Загрузка модели model_1...
Загрузка модели model_2...


### Интерфейс для получения ответа из RAG пайплайна

In [22]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    """
    Функция предобработки текста: приведение к нижнему регистру, удаление лишних символов,
    лемматизация и замена слов по словарю, с сохранением знаков препинания.
    Если слово не найдено в replace_dict, оно остаётся в исходной форме.
    """
    # Токенизация: разделяем слова и знаки препинания
    tokens = re.findall(r'\w+|[^\w\s]', text.lower())
    processed_tokens = []

    for token in tokens:
        if re.match(r'\w+', token):  # Если это слово
            if replace_dict and token in replace_dict:
                # Заменяем слово, если оно есть в словаре
                processed_tokens.append(replace_dict[token])
            else:
                # Если нет в словаре замен, оставляем слово как есть
                processed_tokens.append(token)
        else:
            # Знак препинания остаётся без изменений
            processed_tokens.append(token)

    # Восстановление текста с правильным расположением знаков препинания
    processed_text = ''
    for i, token in enumerate(processed_tokens):
        if token in '.,!?;:':
            # Прикрепляем знак препинания к предыдущему слову без пробела
            processed_text = processed_text.rstrip() + token + ' '
        else:
            processed_text += token + ' '

    return processed_text.strip()

In [23]:
import re
import time
import pandas as pd

def get_answer_from_rag(
    question: str,
    basic_rag_pipeline,
    threshold: float = 0.35,
    df:pd.DataFrame = final_df,
    ):
    start_time = time.time()

    question = preprocess_text(question)
    print(f"Вопрос после очистки: {question}")

    response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "ranker": {"query": question}})
    print(response)

    top_documents = response['ranker']['documents']

    # Проверяем источник первого документа
    first_doc_source = top_documents[0].meta.get('source', '')

    if first_doc_source != "FAQ" or top_documents[0].score < threshold:
        modified_documents = []
        for doc in top_documents:
            if doc.meta.get('source') == "FAQ":
                target_q = doc.content
                # Поиск соответствующего ответа из БЗ
                matched_rows = df.loc[df['Вопрос'] == target_q]
                if not matched_rows.empty:
                    answer_bz = matched_rows.iloc[0]['Ответ из БЗ']
                    doc.content = answer_bz
                    modified_documents.append(doc)
                else:
                    # Если соответствующего вопроса нет в БЗ, оставляем оригинальное содержимое
                    modified_documents.append(doc)
            else:
                modified_documents.append(doc)

        print("Затраченное время:", time.time() - start_time)
        # Возвращаем список модифицированных документов
        return modified_documents
    else:
        # Существующая логика, когда источник первого документа "FAQ"
        document = top_documents[0]

        target_q = document.content
        score = document.score

        if score < threshold:
            answer_text = "Ответ не найден."

            class_1 = classify_question(question, "model_1")
            class_2 = classify_question(question, "model_2")

        elif document.meta.get('source') in ("Генеральное пользовательское соглашение", "Условия размещение контента"):
            answer_text = f"{document.content}\nИсточник: {document.meta.get('source', '')} {document.meta.get('idx', '')}"

            class_1 = classify_question(question, "model_1")
            class_2 = classify_question(question, "model_2")
        else:
            # Предполагается, что 'Вопрос' уникален в df
            matched_rows = df.loc[df['Вопрос'] == target_q]
            if not matched_rows.empty:
                target_answer = matched_rows.iloc[0]
                answer_text = target_answer['Ответ из БЗ']

                class_1 = target_answer.get('Классификатор 1 уровня', None)
                class_2 = target_answer.get('Классификатор 2 уровня', None)
            else:
                answer_text = "Ответ не найден."
                class_1 = classify_question(question, "model_1")
                class_2 = classify_question(question, "model_2")

        print("Затраченное время:", time.time() - start_time)

        return answer_text, class_1, class_2, score

In [24]:
import json
from typing import List
from pydantic import BaseModel

class Answer(BaseModel):
    answer: str
    class_1: str
    class_2: str
    score: float
    source: List

def process_rag_response(question, basic_rag_pipeline, pipe, threshold=0.5, df=final_df, GROUNDED_SYSTEM_PROMPT="Your task is to answer the user's questions using only the information from the provided documents. If the question has nothing to do with the context, just say that you have \"Ответ не найден\"."):
    response = get_answer_from_rag(
        question=question,
        basic_rag_pipeline=basic_rag_pipeline,
        threshold=threshold,
        df=df
    )

    # Проверяем, является ли response кортежем
    if isinstance(response, tuple):
        answer_text, class_1, class_2, score = response
        return Answer(
            answer=answer_text,
            class_1=class_1,
            class_2=class_2,
            score=score,
            source=[{"type": "FAQ", "text": ""}]
        )

    # Проверяем, является ли response списком
    elif isinstance(response, list):
        source = []
        # Предполагается, что список содержит документы
        formatted_documents = []
        for idx, doc in enumerate(response):  # Берём первые три документа
            # Извлекаем заголовок из метаданных, если доступно
            title = doc.meta.get('title', f"Документ {idx + 1}")
            content = doc.content

            # Формируем словарь документа
            formatted_documents.append({
                "doc_id": idx,
                "title": title,
                "content": content
            })
            source.append( {
                "type": doc.meta['source'],
                "text": doc.meta['idx']
            })

        # Создаём sample_history для передачи в LLM модель
        sample_history = [
            {'role': 'system', 'content': GROUNDED_SYSTEM_PROMPT},
            {'role': 'documents', 'content': json.dumps(formatted_documents, ensure_ascii=False)},
            {'role': 'user', 'content': question}
        ]

        # Передаём sample_history в LLM модель и получаем ответ
        answer = pipe(sample_history)
        answer_text = answer[0]['generated_text'][-1]['content']

        return Answer(
            answer=answer_text,
            class_1="ОТСУТСТВУЕТ",
            class_2="Отсутствует",
            score=response[0].score,
            source=source)

### Асинхронная функция для будущего использования с API

In [25]:
import asyncio

async def get_answer(question: str, basic_rag_pipeline) -> Answer:
    """
    Асинхронная функция принимает вопрос, получает ответ и классы с помощью RAG pipeline
    и возвращает объект AnswerModel.

    Параметры:
    - question (str): Вопрос, который нужно обработать.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - replace_dict (dict, optional): Словарь для замены слов в вопросе.
    - df (pd.DataFrame, optional): DataFrame, содержащий столбцы 'Вопрос из БЗ', 'Ответ из БЗ',
                                    'Классификатор 1 уровня' и 'Классификатор 2 уровня'.

    Возвращает:
    - AnswerModel: Объект с полями answer, class_1 и class_2.
    """
    loop = asyncio.get_event_loop()
    curr_answer = await loop.run_in_executor(
        None,
        process_rag_response,
        question,
        basic_rag_pipeline,
        pipe
    )

    return curr_answer

In [26]:
question = "Что администрация может делать?"

my_answer = await get_answer(
    question,
    basic_rag_pipeline
)

my_answer

Вопрос после очистки: что администрация может делать?


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ranker': {'documents': [Document(id=7e6b84aefd7863a16c9cf665f028d225c2bf4832f57c0ddb4fe4a60f5c25dd79, content: 'При обращении Пользователя к Администрации, Администрация вправе запросить у Пользователя документы,...', meta: {'source': 'Agreement', 'idx': '1.7.'}, score: 0.6717730164527893), Document(id=c84cbe11323545feb00fd4363429c97de28c626ae4f0a4c5a6d55eadf81ac441, content: 'сколько человек можно сделать администраторами?', meta: {'source': 'FAQ', 'idx': 525}, score: 0.547361433506012), Document(id=0212788db2e1530325d0f6e090eed2555104be7a5720408a18adae4cea9d72f5, content: 'Какие возможности взаимодействия со Студией доступны пользователю с правами Редактора?', meta: {'source': 'FAQ', 'idx': 93}, score: 0.15697705745697021)]}}
Затраченное время: 39.700140714645386


Answer(answer='Администрация имеет право на запрос у Пользователя документов, подтверждающих информацию, указанную в обращении, и право Пользователя на направление такого обращения. Если Пользователь не предоставит запрошенные документы или информацию, Администрация может отказаться от выполнения каких-либо действий, связанных с обращением Пользователя.', class_1='ОТСУТСТВУЕТ', class_2='Отсутствует', score=0.6717730164527893, source=[{'type': 'Agreement', 'text': '1.7.'}, {'type': 'FAQ', 'text': 525}, {'type': 'FAQ', 'text': 93}])

### Тест модели

In [27]:
import pandas as pd

def process_all_questions(input_file, question_col, basic_rag_pipeline, output_file):
    """
    Функция обрабатывает все вопросы из указанного файла, вызывает get_answer_from_rag для каждого,
    и сохраняет результаты в новый CSV файл.

    Параметры:
    - input_file: Путь к CSV или XLSX файлу с вопросами.
    - question_col: Название колонки, содержащей вопросы.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - output_file: Путь для сохранения результирующего CSV файла с вопросами и ответами.

    Возвращает:
    - DataFrame с исходными вопросами и найденными ответами.
    """

    file_extension = input_file.split('.')[-1].lower()

    # Чтение данных в зависимости от расширения
    if file_extension == 'csv':
        df_questions = pd.read_csv(input_file)
    elif file_extension == 'xlsx':
        df_questions = pd.read_excel(input_file)
    else:
        raise ValueError("Файл должен быть формата CSV или XLSX.")

    results = []

    for question in df_questions[question_col]:
        answer, class_1, class_2 = process_rag_response(question, basic_rag_pipeline, pipe)
        results.append({'Вопрос': question, 'Ответ': answer})

    df_results = pd.DataFrame(results)
    df_results.to_csv(output_file, index=False, encoding='utf-8')

    print(f"Результаты сохранены в файл: {output_file}")
    return df_results

In [28]:
# main_path = "drive/MyDrive/HacksAI/Russia24/"

# src_file_path = main_path + "ground_truth_data/pipeline3_g_t_data.csv"
# out_file_path = main_path + "generated_data/pipeline3_answers.csv"

# process_all_questions(src_file_path, 'Вопрос', basic_rag_pipeline, out_file_path)

# Телеграм бот

In [29]:
# @title Установка зависимостей бота

!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.7/588.7 kB 35.6 MB/s eta 0:00:00


In [ ]:
# @title Код бота

import asyncio
import logging
import os

import sys
import yaml
from aiogram import Bot, Dispatcher
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.filters import CommandStart
from aiogram.types import Message, InlineKeyboardButton, InlineKeyboardMarkup, CallbackQuery

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

NO_ANSWER = "Ответ не найден."

SCORE_THRESHOLD = float(os.getenv('SCORE_THRESHOLD', '0.75'))

# Загрузка сообщений бота из файла
BOT_MESSAGES_FILE_PATH = '/content/drive/MyDrive/HacksAI/Russia24/bot-messages.yml'
with open(BOT_MESSAGES_FILE_PATH, 'r', encoding='utf-8') as file:
    bot_messages = yaml.safe_load(file)

# Инициализация диспетчера
dp = Dispatcher()


# Команда /start для приветствия
@dp.message(CommandStart())
async def command_start_handler(message: Message) -> None:
    await message.reply(bot_messages['start'])


# Обработчик лайка
@dp.callback_query(lambda query: query.data.startswith('like:'))
async def like_handler(query: CallbackQuery):
    await query.answer(bot_messages['like'])


# Обработчик дизлайка
@dp.callback_query(lambda query: query.data.startswith('dislike:'))
async def dislike_handler(query: CallbackQuery):
    await query.answer(bot_messages['dislike'])


# Обработчик вопросов
@dp.message()
async def question_handler(message: Message) -> None:
    try:
        question = message.text
        answer_data = await get_answer(
            question,
            basic_rag_pipeline
        )
        if answer_data.answer == NO_ANSWER:
            await message.reply(bot_messages['answer-no'])
            return
        text = get_answer_text(answer_data, False)
        markup = create_answer_markup()
        await message.reply(text, reply_markup=markup)
    except Exception as exception:
        error_text = bot_messages['error'].format(exception=str(exception) if verbose else '')
        await message.reply(error_text)


# Создание кнопок для сообщения

def create_answer_markup() -> InlineKeyboardMarkup:
    return InlineKeyboardMarkup(inline_keyboard=[[
        InlineKeyboardButton(text="👍", callback_data=f'like:fake'),
        InlineKeyboardButton(text="👎", callback_data=f'dislike:fake')
    ]])


# Вспомогательные методы

def get_verbose_status(verbose: bool) -> str:
    return bot_messages['verbose-enabled'] if verbose else bot_messages['verbose-disabled']


def get_answer_text(response, verbose: bool) -> str:
    confidence = response.score >= SCORE_THRESHOLD
    assurance_text = bot_messages['answer-confident'] if confidence else bot_messages['answer-doubtful']
    docs_text = get_docs_text(response.source)
    other_text = response.get_other_inline() if verbose else ''
    return bot_messages['answer'].format(
        answer=response.answer,
        class_1=response.class_1,
        class_2=response.class_2,
        assurance=assurance_text,
        docs=docs_text,
        other=other_text
    )


def get_docs_text(source):
    unique_types = set()
    agreement_texts = []
    for item in source:
        if item['type'] == 'Agreement':
            agreement_texts.append(f"Генеральное Пользовательское Соглашение RUTUBE {item['text']}")
        else:
            unique_types.add(item['type'])
    result = '\n'.join(unique_types)
    if agreement_texts:
        result = '\n'.join(agreement_texts) + '\n' + result
    return result


async def main() -> None:
    bot_token = '7622913538:AAH5gBxTuRqpDFGRsaTDVz2TD_dQ5wBahOE'
    bot = Bot(token=bot_token, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
    await dp.start_polling(bot)


await main()


Вопрос после очистки: ты тут?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ranker': {'documents': [Document(id=a13ad1c4bd0f977cd1de6fac7b5620f2fe61a1096be145fe528e90e313bc8cfd, content: 'Настоящее Соглашение является смешанным договором по смыслу п. 2 и п.3 ст. 421 ГК РФ.', meta: {'source': 'Agreement', 'idx': '2.2.'}, score: 0.15427424013614655), Document(id=0f93fd09e838a95e32308cee9063de567394a1960881b24c03c9d7500e310ce5, content: 'Пользователю в этой роли доступен только раздел "Комментарии": https://studio.rutube.ru/comments, гд...', meta: {'source': 'FAQ', 'idx': 1226}, score: 0.08525286614894867), Document(id=ae7d6242d5a557f5b661494180bd4b4375c7a1a02b91fd20c53d30d3a0378820, content: 'какие доступы есть в студии?', meta: {'source': 'FAQ', 'idx': 547}, score: 0.0563422255218029)]}}
Затраченное время: 0.5483376979827881
Вопрос после очистки: как я обязан использовать rutube?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ranker': {'documents': [Document(id=15909904fbb6d432bd3abfd90f1f249e512a6a87b35948c1e2ff05217e92ab65, content: 'Пользователь обязуется использовать RUTUBE на условиях, определяемых настоящим Соглашением и Админис...', meta: {'source': 'Agreement', 'idx': '1.2.'}, score: 0.9713512659072876), Document(id=49c905c88c51bbf335e71012003ea8feb7eb34513d2420f99cc6e2fbea1d8279, content: 'Использование RUTUBE в целом, а также Информация и Контент, размещаемые на RUTUBE, действия и сообще...', meta: {'source': 'Agreement', 'idx': '4.25.'}, score: 0.5180403590202332), Document(id=2af3a3852d859ecb71ae44090a7030ad2020c0575199aae388db5a8940901b0d, content: 'Авторизуйтесь на RUTUBE, перейдите в свой профиль: https://rutube.ru/profile, нажмите «Изменить паро...', meta: {'source': 'FAQ', 'idx': 1363}, score: 0.3671049475669861)]}}
Затраченное время: 0.3782169818878174
Вопрос после очистки: как подрубить стримчанский?


...

{'ranker': {'documents': [Document(id=2e9c10585840ee3834aa15a8ff10390997f34596bf138bb151b820a318ed9a68, content: 'Что такое личный канал?', meta: {'source': 'FAQ', 'idx': 933}, score: 0.19142864644527435), Document(id=0fac3735405ad0d2e6f8d3a1c2e56ff2c69501e1f6c55727b6e1bfaa0ace54b0, content: 'Что за личный канал?', meta: {'source': 'FAQ', 'idx': 931}, score: 0.027118109166622162), Document(id=07e9d04a87ddded650bdfcfd9d8ddf702c99eeff96b35fd56a2de784a0603eab, content: 'Почему некоторые видео называются "shorts", что это?', meta: {'source': 'FAQ', 'idx': 584}, score: 0.02274135686457157)]}}
Затраченное время: 0.47382211685180664
Вопрос после очистки: где ознакомиться с соглашением пользователя?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ranker': {'documents': [Document(id=bc0b9e4942be11175e1e62f9b7f4d74fd3fad06dc32ed5d89e7923748a0d6b72, content: 'Соглашение вступает в силу с момента его принятия Пользователем и действует в течение неопределенног...', meta: {'source': 'Agreement', 'idx': '12.7.'}, score: 0.15581761300563812), Document(id=9cfb1e3d8aeb261dd4727b28d2e4f05446db40094eb9c7d3704e04484b473aec, content: 'как связаться с поддержкой в приле студии?', meta: {'source': 'FAQ', 'idx': 427}, score: 0.035741496831178665), Document(id=3bc3e34593472ae0f9ffebfff240276cce26fe65fdc5844e8cf5d6e86ea315ef, content: 'Какие возможности взаимодействия со Студией доступны пользователю в роли Пользователя с правами прос...', meta: {'source': 'FAQ', 'idx': 101}, score: 0.006674938835203648)]}}
Затраченное время: 0.30460333824157715
Вопрос после очистки: что такое соглашение?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ranker': {'documents': [Document(id=a13ad1c4bd0f977cd1de6fac7b5620f2fe61a1096be145fe528e90e313bc8cfd, content: 'Настоящее Соглашение является смешанным договором по смыслу п. 2 и п.3 ст. 421 ГК РФ.', meta: {'source': 'Agreement', 'idx': '2.2.'}, score: 0.4567139744758606), Document(id=bc0b9e4942be11175e1e62f9b7f4d74fd3fad06dc32ed5d89e7923748a0d6b72, content: 'Соглашение вступает в силу с момента его принятия Пользователем и действует в течение неопределенног...', meta: {'source': 'Agreement', 'idx': '12.7.'}, score: 0.16828936338424683), Document(id=91f4344ec37fa728c57c2bfd9b23138617a0e810fb5caef39521a062f008dfd0, content: 'Что за можеерация?', meta: {'source': 'FAQ', 'idx': 884}, score: 0.018764806911349297)]}}
Затраченное время: 0.29454660415649414
